In [29]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import UserBasedRecommender as UBR
import random
from scipy import sparse as sps
import importlib

In [30]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_playlists = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tracks = pd.read_csv('Data/target_tracks.csv','\t')

In [31]:
train, test, tgt_tracks, tgt_playlists = rs.train_test_split_interface(train, 10, 20, 5, 2517)
tgt_playlists.shape

(9129, 1)

In [32]:
rs = importlib.reload(rs)
ibr = IBR.ItemBasedRecommender(idf=True, shrinkage=10,n_el_sim=15)
ibr.fit(tracks,train,tgt_tracks, multiprocessing=True)#, saved_similarity='BuiltStructures/ibr_sim_65el_h10_idfTrue.npz')
print('Model fitted!')

Calculated Indices
Model URM built
Model URM regularized with IDF!
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 20000 similariti

In [33]:
rs = importlib.reload(rs)
tsr= TSR.TopSimilarRecommender(attributes=['artist_id', 'album'],idf=True, n_min_attr=90, n_el_sim=120)
tsr.fit(tracks,tgt_tracks,multiprocessing=True)#, saved_similarity='BuiltStructures/tsr_sim_65el_idfTrue_artist_album_playcount.npz')
print('Model fitted!')

Fixed dataset
Calculated Indices
ICM built
ICM regularized with IDF!
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 20000 similari

In [34]:
rs = importlib.reload(rs)
UBR = importlib.reload(UBR)
ubr = UBR.UserBasedRecommender(measure='imp_cos', shrinkage=10,n_el_sim =15)
ubr.fit(tracks,train,tgt_playlists,multiprocessing=True)#, saved_similarity='BuiltStructures/ubr_sim_65el_impcos.npz')
print('Model fitted!')

Calculated Indices
(45649, 100000)
Model URM built
Computed 0 similarities over chunk of 11412 elements.
Computed 0 similarities over chunk of 11412 elements.
Computed 0 similarities over chunk of 11413 elements.
Computed 0 similarities over chunk of 11412 elements.
Computed 5000 similarities over chunk of 11413 elements.
Computed 5000 similarities over chunk of 11412 elements.
Computed 5000 similarities over chunk of 11412 elements.
Computed 5000 similarities over chunk of 11412 elements.
Computed 10000 similarities over chunk of 11412 elements.
Computed 10000 similarities over chunk of 11413 elements.
Computed 10000 similarities over chunk of 11412 elements.
Computed 10000 similarities over chunk of 11412 elements.
Similarity built
Model fitted!


In [35]:
# CBF with tags
rs = importlib.reload(rs)
tsr_tag= TSR.TopSimilarRecommender(attributes=['artist_id','album','tags'],idf=True, n_min_attr=90, n_el_sim=120)
tsr_tag.fit(tracks,tgt_tracks,multiprocessing=True)#, save_sim=True)
print('Model fitted!')

Fixed dataset
Calculated Indices
ICM built
ICM regularized with IDF!
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 0 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 5000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 10000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 15000 similarities over chunk of 25000 elements.
Computed 20000 similari

# Recommendation

In [36]:
rs = importlib.reload(rs)
_, _, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
_, _, IX_playlists, _ = rs.create_sparse_indexes(tracks_info=tracks, playlists=train, tracks_reduced=tracks)
URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM_UBR =rs.create_UBR_URM(IX_playlists, tsr.IX_tgt_items, train)
print('URM built')

URM built


In [37]:
ubr.S =ubr.S.T.tocsr()
URM = URM.tocsr()
tsr.S = tsr.S.tocsr()
tsr_tag.S = tsr_tag.S.tocsr()
ibr.S =ibr.S.tocsr()
URM_UBR= URM_UBR.tocsr()

In [38]:
div_t = tsr.S.sum(axis=0)
div_i = ibr.S.sum(axis=0)
div_tt = tsr_tag.S.sum(axis=0)
div_t.shape

(1, 30319)

In [39]:
rs = importlib.reload(rs)
recommendetions = np.array([])
H_t=60
H_i=200
H_tt=60
for p in IX_tgt_playlists.values:
    avg_sims_t = URM[p,:].dot(tsr.S).toarray().ravel()
    avg_sims_t = np.array(avg_sims_t/(div_t+H_t+1e-6)).ravel()
    #avg_sims_tt = URM[p,:].dot(tsr_tag.S).toarray().ravel()
    #avg_sims_tt = avg_sims_tt/(div_tt+H_tt+1e-6)
    avg_sims_i = URM[p,:].dot(ibr.S).toarray().ravel()
    avg_sims_i = np.array(avg_sims_i/(div_i+H_i+1e-6)).ravel()
    avg_sims_u = ubr.S[p,:].dot(URM_UBR).toarray().ravel()
    #avg_sims = np.array(avg_sims_t*1 + avg_sims_i*0+avg_sims_u*0).ravel()
    top = rs.top5_outside_playlist(avg_sims_t, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))
rec_tsr =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

Recommended 0 users over 9129
Recommended 1000 users over 9129
Recommended 2000 users over 9129
Recommended 3000 users over 9129
Recommended 4000 users over 9129
Recommended 5000 users over 9129
Recommended 6000 users over 9129
Recommended 7000 users over 9129
Recommended 8000 users over 9129
Recommended 9000 users over 9129


# Evaluation

In [40]:
map_eval = rs.evaluate(rec_tsr, test, 'MAP')
print('Evaluation completed!')

Evaluation completed!


In [41]:
map_eval

0.053651002300361746

# Save to csv

In [13]:
rec_tsr.to_csv('Submissions/ensemble_submission_notags_alpha0304015015_h15_30_15_division_calculated_prev.csv', index=False)
print('Results saved as csv!')

Results saved as csv!
